# Web scraper

Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as np
import pandas as pd
from re import search
from re import match
import sys
from IPython.display import display

Define the search query and create the driver

In [2]:
#Omistusasunnot Porvoossa sivulla asunnot.oikotie.fi
search_query = 'https://asunnot.oikotie.fi/myytavat-asunnot/porvoo?pagination=1&locations=%5B%5B319,6,%22Porvoo%22%5D%5D&cardType=100&habitationType%5B%5D=1'
driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application/chromedriver.exe')

Click through a popup and click on the first listings link

In [ ]:
driver.get(search_query)

#Odotetaan evästeiden hyväksyntä iframe esiintymistä ja vaihdetaan siihen
driver.implicitly_wait(10)
driver.switch_to.frame(1)

#Hyväksytään ja palataan takaisin 
driver.find_element_by_xpath("//*[@title = 'Hyväksy kaikki evästeet']").click()
driver.switch_to.default_content()
driver.implicitly_wait(10)

#Etsitään ensimmäisen asunnon linkki ja edetään sen sivulle
first_page = driver.find_element_by_xpath("//a[contains(@analytics-click-position, '1')]")
first_page.click()
driver.implicitly_wait(10)

Store the number of pages

In [ ]:
#Etsitään sivulta kaikkien asuntojen määrä ja otetaan se talteen
pages_element = driver.find_element_by_xpath("/html/body/main/div[5]/div/div[2]/div[2]/result-browse/div/span").get_attribute("innerHTML")
pages = int(pages_element.replace('1 /', ''))
driver.implicitly_wait(10)

Loop through all of the pages and store data of interest into lists

In [ ]:
#Sijoitetaan tänne kaikki dict muotoiset listaukset. Luodaan tästä lopulta dataframe
listings = []

for i in range(pages):
    try:
        #Määritellään listat, joihin sijoitetaan löydetyt tiedot
        descriptions = driver.find_elements_by_xpath("//p[contains(@class, 'paragraph paragraph--keep-formatting')]")
        titles = driver.find_elements_by_xpath("//dt[contains(@class, 'info-table__title')]")
        values = driver.find_elements_by_xpath("//dd[contains(@class, 'info-table__value')]")

        #Alustetaan listat
        desc_list = []
        title_list = []
        value_list = []
        
        #Käydään läpi kuvaukset ja yhdistetään kappaleet
        for desc in descriptions:
            desc_list.append(desc.get_attribute('innerHTML'))
            description = ' '.join(desc_list)
        #Otetaan talteen dataframe sarakkeiden nimet listaan
        for title in titles:
            title_list.append(title.get_attribute('innerHTML'))
        #Otetaan talteen dataframe sarakkeiden arvot listaan
        #Kohta "Tarkista uuden tai nykyisen kotisi kotivakuutuksen hinta ja ehdot" ei sisällä otsikkoa, joten jätetään se pois
        for value in values:
            if not search("Tarkista uuden tai nykyisen kotisi kotivakuutuksen hinta ja ehdot", value.get_attribute('innerHTML')):
                value_list.append(value.get_attribute('innerHTML'))
        #Luodaan sanakirja, joka edustaa yksittäis asunto oliota        
        zip_iter = zip(title_list, value_list)
        listing = dict(zip_iter)
        #Lisätään kuvaus sarake ja listään kuvaukset siihen
        listing['Kuvaus'] = description
        listings.append(listing)
        #Jos kyseessä on viimeinen sivu, niin lopetetaan loop
        if i == pages - 1:
            break
        driver.find_element_by_xpath("//a[contains(@analytics-click-label, 'next')]").click()
    except Exception as e:
        print(e.__class__)
        driver.quit()
        sys.exit()

Create the dataframe and make a .csv

In [ ]:
df = pd.DataFrame(listings)
display(df.head())
df.to_csv("porvoo_listings22.csv")
driver.quit()